In [6]:
from typing_extensions import TypedDict, Annotated, List, Literal
from langchain_core.messages import AnyMessage, AIMessage, HumanMessage, SystemMessage
from langgraph.graph import add_messages,StateGraph, START, END
from langgraph.graph.message import add_messages 
from typing import Dict, List, Optional, Union, Literal
from pydantic import BaseModel, Field,field_validator
from IPython.display import Image, display 
from langchain_core.messages import AnyMessage
from enum import Enum

In [17]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [19]:
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ['GROQ_API_KEY'] = os.getenv("GROQ_API_KEY")
os.environ['LANGSMITH_API_KEY'] = os.getenv("LANGCHAIN_API_KEY")
os.environ['LANGSMITH_PROJECT'] = os.getenv("LANGCHAIN_PROJECT")
os.environ['LANGSMITH_TRACING'] = "True"

In [13]:
class TechnicalFeasibilityEnum(str, Enum):
    FEASIBLE = 'feasible'
    NOT_FEASIBLE = 'not_feasible'
    PARTIALLY_FEASIBLE = 'partially_feasible'

class EconomicFeasibilityEnum(str, Enum):
    VIABLE = 'viable'
    NOT_VIABLE = 'not_viable'
    MARGINAL = 'marginal'


class FeasibilityStudySchema(BaseModel):
    
    technical_feasibility: TechnicalFeasibilityEnum = Field(
        description='Assessment of whether the project can be built with available technology and skills.'
    )

    technical_details: str = Field(
        default=None,
        description='Detailed explanation of technical feasibility findings, including challenges or required resources.'
    )

    economic_feasibility: EconomicFeasibilityEnum = Field(
        description='Assessment of the financial viability of the project (cost vs. benefit).'
    )

    economic_details: str = Field(
        default=None,
        description='Detailed explanation of economic feasibility findings, including estimated costs, ROI, or funding needs.'
    )

    class Config:
        use_enum_values = True
        json_schema_extra = {
            'example': {
                'technical_feasibility': 'feasible',
                'technical_details': '''Technical assessment shows the project is highly feasible with current resources and technology stack:

                TECHNICAL STRENGTHS:
                • Experienced development team with 5+ years in similar e-commerce projects
                • Proven technology stack: React.js frontend, Node.js backend, PostgreSQL database
                • Existing CI/CD pipeline and AWS infrastructure already in place
                • Team familiar with payment gateway integrations (Stripe, PayPal)
                • Robust authentication and security frameworks available

                TECHNICAL REQUIREMENTS:
                • Need to implement Redis for session management and caching
                • Elasticsearch integration required for advanced product search
                • Mobile responsiveness using existing CSS framework
                • API rate limiting and security middleware implementation

                TECHNICAL CHALLENGES:
                • Real-time inventory management across multiple warehouses
                • Complex product variant system (size, color, price variations)
                • Integration with existing legacy accounting system via APIs
                • Performance optimization for handling 10,000+ concurrent users

                RESOURCE NEEDS:
                • 2 additional senior backend developers for complex integrations
                • 1 frontend specialist for advanced UI components
                • DevOps engineer for scaling infrastructure
                • Estimated 8-10 months development timeline

                RISK MITIGATION:
                • Prototype critical integrations early in development cycle
                • Implement comprehensive automated testing suite
                • Use proven architectural patterns and design principles
                • Regular code reviews and pair programming sessions

                The technical foundation is solid and the team has the necessary expertise to deliver a robust e-commerce platform.''',
                            
                            'economic_feasibility': 'viable',
                            'economic_details': '''Economic analysis demonstrates strong financial viability with positive ROI projected within 18 months:

                DEVELOPMENT COSTS:
                • Team salaries (10 months): $850,000
                - 6 developers  $120k annually  10/12 months = $600k
                - 1 project manager  $100k annually  10/12 months = $83k
                - 1 designer  $90k annually  10/12 months = $75k
                - DevOps consultant $150/hour  400 hours = $60k
                - QA engineer  $80k annually  6/12 months = $40k

                • Infrastructure costs (Year 1): $120,000
                - AWS hosting and services: $8k/month  12 = $96k
                - Third-party services (payment, analytics, CDN): $2k/month 12 = $24k

                • Third-party licenses and tools: $45,000
                - Development tools and software licenses
                - Security and monitoring services
                - Payment gateway setup fees

                • Marketing and launch: $100,000
                - Digital marketing campaigns
                - SEO optimization
                - Launch event and PR

                TOTAL INVESTMENT: $1,115,000

                REVENUE PROJECTIONS:
                • Year 1: $1,400,000 (conservative estimate)
                - Average order value: $75
                - Monthly orders: 1,500 (ramping from 500 to 2,500)
                - 12-month projection with seasonal variations

                • Year 2: $3,200,000 (growth phase)
                - Increased marketing and customer base expansion
                - Higher average order value through cross-selling
                - Mobile app launch driving additional traffic

                • Year 3: $5,800,000 (maturity phase)
                - Market penetration and brand recognition
                - International expansion opportunities
                - Premium product lines introduction

                FINANCIAL METRICS:
                • Break-even point: Month 14
                • ROI Year 1: 25% ($285k profit)
                • ROI Year 2: 187% ($2,085k cumulative profit)
                • ROI Year 3: 420% ($4,685k cumulative profit)

                FUNDING STATUS:
                • Series A funding secured: $1,500,000
                • Additional runway available: $600,000
                • Sufficient capital to cover development and 6 months operations

                MARKET ANALYSIS:
                • Target market size: $50M annually in our segment
                • Competition analysis shows 15-20% market share achievable
                • Growing demand for online shopping in our region (+25% annually)

                The economic fundamentals are strong with multiple revenue streams and clear path to profitability.'''
            }
    } 

In [14]:
class RequirementType(str, Enum):
    FUNCTIONAL = "functional"
    NON_FUNCTIONAL = "non_functional"
    USER_INTERFACE = "user_interface"


class PreferedTechStack(str, Enum):
    MERN_JS = "mern - mongodb, expressjs, reactjs,nodejs using javascript,not typescript"
    MEAN_JS = "marn - mongodb, expressjs, angularjs,nodejs using javascript,not typescript"
    MERN_TS = "mern - mongodb, expressjs, reactjs,nodejs using typescript"
    MEAN_TS = "marn - mongodb, expressjs, angularjs,nodejs using typescript"
    SPRING_BOOT = "Java Spring Boot"

class ProjectDomain(str, Enum):
    FINTECH = "fintech"
    EDTECH = "edtech"
    HEALTHCARE = "healthcare"
    ECOMMERCE = "ecommerce"
    SOCIAL_MEDIA = "social_media"
    PRODUCTIVITY = "productivity"
    OTHER = "other"

class RequirementPriority(str, Enum):
    CRITICAL = "critical"
    HIGH = "high"
    MEDIUM = "medium"
    LOW = "low"
    NICE_TO_HAVE = "nice_to_have"

class RequirementDesc(BaseModel):
    requirement: str = Field(..., description='Description of requirement')
    priority: RequirementPriority = Field(..., description='Priority level of the requirement')
    type:RequirementType = Field(...,description='Type of the requirement')
    
class ProjectRequirmentsSchema(BaseModel):

    title: str = Field(
        ..., min_length=3, max_length=200, description='The title of the project'
    )

    description: str = Field(
        ..., min_length=100, description='Detailed description of the project'
    )

    project_duration_in_weeks:Optional[int] = Field(
        ..., min=1,max=52,description='Duration of project'
    )

    requirements: List[RequirementDesc] = Field(
        ..., 
        description='The list of requirements with their priority levels and type'
    )    

    domain: ProjectDomain = Field(..., description='Domain or category (e.g., fintech, edtech) this project belongs to')
    
    prefered_techstack: PreferedTechStack = Field(...,description='Prefered tech stack of this project')

In [ ]:
FEASIBILITY_STUDY_SYSTEM_PROMPT = """
You are an expert project analyst specializing in project development feasibility studies. 
Your role is to conduct comprehensive technical and economic feasibility assessments for software projects.

## Assessment Framework

### Technical Feasibility Assessment
Evaluate the project's technical viability considering:

**Technology Stack Analysis:**
- Assess compatibility and maturity of the proposed technology stack
- Evaluate team expertise requirements and learning curve
- Identify technical dependencies and third-party integrations
- Review scalability and performance requirements

**Resource Evaluation:**
- Analyze required development team size and skill sets
- Assess infrastructure and hosting requirements
- Evaluate development tools and environment needs
- Identify knowledge gaps and training requirements

**Risk Assessment:**
- Identify major technical challenges and obstacles
- Evaluate complexity of critical and high-priority requirements
- Assess integration complexity with existing systems
- Review security and compliance requirements

**Implementation Considerations:**
- Evaluate development timeline feasibility
- Assess testing and quality assurance requirements
- Review deployment and maintenance considerations
- Identify potential bottlenecks and mitigation strategies

### Economic Feasibility Assessment
Analyze the project's financial viability:

**Cost Analysis:**
- Calculate development costs (team salaries, tools, infrastructure)
- Estimate operational costs (hosting, maintenance, support)
- Factor in domain-specific compliance and security costs
- Include contingency budget (15-20% of total costs)

**Revenue Projections:**
- Analyze market size and target audience
- Project revenue streams and pricing models
- Estimate user adoption and growth rates
- Consider market trends and seasonal variations

**Financial Metrics:**
- Calculate Return on Investment (ROI)
- Determine break-even timeline
- Assess funding requirements
- Evaluate long-term profitability potential

## Classification System

### Technical Feasibility Classification:
- **feasible**: Project can be successfully built with available resources and technology
- **partially_feasible**: Project is achievable but requires significant additional resources or has notable constraints
- **not_feasible**: Project cannot be realistically completed with current constraints

### Economic Feasibility Classification:
- **viable**: Project shows strong financial potential with positive ROI
- **marginal**: Project may be financially viable but carries significant risk or minimal returns
- **not_viable**: Project is not financially sustainable or profitable

## Required Analysis Components

**Technical Details Must Include:**
- Comprehensive assessment of technical strengths and capabilities
- Detailed technical requirements and dependencies
- Specific challenges and potential obstacles
- Resource needs (team size, skills, tools, infrastructure)
- Risk mitigation strategies for critical requirements
- Realistic timeline breakdown and milestones
- Performance and scalability considerations
- Integration complexity analysis

**Economic Details Must Include:**
- Itemized development and operational costs breakdown
- Infrastructure and hosting cost estimates
- Market analysis and revenue projections
- Key financial metrics (ROI, break-even timeline, profitability)
- Funding requirements and cash flow analysis
- Market size and competitive analysis
- Risk factors and sensitivity analysis
- Multi-year financial projection

## Analysis Guidelines:
1. Base assessments on realistic assumptions and industry standards
2. Consider technology-specific development challenges and benefits
3. Factor in timeline constraints and resource availability
4. Prioritize critical and high-priority requirements in analysis
5. Provide actionable insights for decision-making
6. Use concrete numbers and specific examples where possible
7. Address both optimistic and conservative scenarios
8. Maintain objectivity and professional judgment

Generate a comprehensive feasibility study that addresses all technical and economic aspects. The analysis should be detailed, realistic, and provide clear recommendations for project approval or rejection based on the feasibility assessment.
"""

def format_feasibility_prompt(project_requirements:ProjectRequirmentsSchema):
    """
    Format the feasibility study prompt with project requirements data
    
    Args:
        project_requirements: ProjectRequirmentsSchema object
        
    Returns:
        str: Formatted prompt ready for LLM
    """
    
    # Format requirements list
    requirements_list = ""
    for i, req in enumerate(project_requirements.requirements, 1):
        requirements_list += f"{i}. **{req.priority.upper()} - {req.type.upper()}**: {req.requirement}\n"
    
    return FEASIBILITY_STUDY_SYSTEM_PROMPT.format(
        title=project_requirements.title,
        description=project_requirements.description,
        duration=project_requirements.project_duration_in_weeks,
        domain=project_requirements.domain,
        tech_stack=project_requirements.prefered_techstack,
        requirements_list=requirements_list
    )

# Example usage in your LangGraph node:
def feasibility_study_node(state: ProjectWorkflowState):
    """
    Generate feasibility study based on project requirements
    """
    project_reqs = state.get('project_requirements')
    
    if not project_reqs:
        raise ValueError("Project requirements not found in state")
    
    # Format the prompt
    formatted_prompt = format_feasibility_prompt(project_reqs)
    
    # Here you would call your LLM with the formatted prompt
    # llm_response = your_llm.invoke(formatted_prompt)
    
    # Parse the response into FeasibilityStudySchema
    # feasibility_study = parse_feasibility_response(llm_response)
    
    return {
        "current_stage": "feasibility_analysis",
        "feasibility_study": None,  # Replace with parsed response
        "feasibility_json": "",     # Replace with JSON output
        "feasibility_approved": False  # Set based on analysis
    }

# Additional prompt for feasibility revision
FEASIBILITY_REVISION_PROMPT = """
You are reviewing and revising a feasibility study based on stakeholder feedback and updated requirements.

## Original Feasibility Study:
{original_study}

## Revision Requirements:
{revision_feedback}

## Updated Project Requirements (if any):
{updated_requirements}

Please revise the feasibility study addressing the following:
1. Incorporate stakeholder feedback and concerns
2. Update technical assessment based on new information
3. Revise economic analysis with updated assumptions
4. Maintain consistency with original project goals
5. Provide clear explanation of changes made

Generate the revised feasibility study maintaining the same structure and classification system:
- Technical Feasibility: feasible/partially_feasible/not_feasible
- Economic Feasibility: viable/marginal/not_viable

Focus on areas specifically mentioned in the revision feedback while maintaining overall study quality and accuracy.
"""

def format_revision_prompt(original_study, feedback, updated_reqs=None):
    """
    Format prompt for feasibility study revision
    
    Args:
        original_study: Original FeasibilityStudySchema object
        feedback: Stakeholder feedback string
        updated_reqs: Updated requirements (optional)
        
    Returns:
        str: Formatted revision prompt
    """
    return FEASIBILITY_REVISION_PROMPT.format(
        original_study=original_study.model_dump_json(indent=2) if original_study else "No original study provided",
        revision_feedback=feedback,
        updated_requirements=updated_reqs or "No updated requirements"
    )

In [21]:
from langchain_groq import ChatGroq 
llm = ChatGroq(model="deepseek-r1-distill-llama-70b")

In [20]:
from langchain_core.prompts import PromptTemplate 
from langchain_core.output_parsers import JsonOutputParser 
output_parser = JsonOutputParser()

prompt = PromptTemplate(
    template="{system_prompt} \n {format_instruction} \n {human_query} \n",
    input_variables= ["system_prompt", "human_query",],
    partial_variables={"format_instruction" : output_parser.get_format_instructions()}
)

In [22]:
FEASIBILITY_STUDY_SYSTEM_PROMPT = """
You are an expert project analyst specializing in project development feasibility studies. 
Your role is to conduct comprehensive technical and economic feasibility assessments for software projects.

## Assessment Framework

### Technical Feasibility Assessment
Evaluate the project's technical viability considering:

**Technology Stack Analysis:**
- Assess compatibility and maturity of the proposed technology stack
- Evaluate team expertise requirements and learning curve
- Identify technical dependencies and third-party integrations
- Review scalability and performance requirements

**Resource Evaluation:**
- Analyze required development team size and skill sets
- Assess infrastructure and hosting requirements
- Evaluate development tools and environment needs
- Identify knowledge gaps and training requirements

**Risk Assessment:**
- Identify major technical challenges and obstacles
- Evaluate complexity of critical and high-priority requirements
- Assess integration complexity with existing systems
- Review security and compliance requirements

**Implementation Considerations:**
- Evaluate development timeline feasibility
- Assess testing and quality assurance requirements
- Review deployment and maintenance considerations
- Identify potential bottlenecks and mitigation strategies

### Economic Feasibility Assessment
Analyze the project's financial viability:

**Cost Analysis:**
- Calculate development costs (team salaries, tools, infrastructure)
- Estimate operational costs (hosting, maintenance, support)
- Factor in domain-specific compliance and security costs
- Include contingency budget (15-20% of total costs)

**Revenue Projections:**
- Analyze market size and target audience
- Project revenue streams and pricing models
- Estimate user adoption and growth rates
- Consider market trends and seasonal variations

**Financial Metrics:**
- Calculate Return on Investment (ROI)
- Determine break-even timeline
- Assess funding requirements
- Evaluate long-term profitability potential

## Classification System

### Technical Feasibility Classification:
- **feasible**: Project can be successfully built with available resources and technology
- **partially_feasible**: Project is achievable but requires significant additional resources or has notable constraints
- **not_feasible**: Project cannot be realistically completed with current constraints

### Economic Feasibility Classification:
- **viable**: Project shows strong financial potential with positive ROI
- **marginal**: Project may be financially viable but carries significant risk or minimal returns
- **not_viable**: Project is not financially sustainable or profitable

## Required Analysis Components

**Technical Details Must Include:**
- Comprehensive assessment of technical strengths and capabilities
- Detailed technical requirements and dependencies
- Specific challenges and potential obstacles
- Resource needs (team size, skills, tools, infrastructure)
- Risk mitigation strategies for critical requirements
- Realistic timeline breakdown and milestones
- Performance and scalability considerations
- Integration complexity analysis

**Economic Details Must Include:**
- Itemized development and operational costs breakdown
- Infrastructure and hosting cost estimates
- Market analysis and revenue projections
- Key financial metrics (ROI, break-even timeline, profitability)
- Funding requirements and cash flow analysis
- Market size and competitive analysis
- Risk factors and sensitivity analysis
- Multi-year financial projection

## Analysis Guidelines:
1. Base assessments on realistic assumptions and industry standards
2. Consider technology-specific development challenges and benefits
3. Factor in timeline constraints and resource availability
4. Prioritize critical and high-priority requirements in analysis
5. Provide actionable insights for decision-making
6. Use concrete numbers and specific examples where possible
7. Address both optimistic and conservative scenarios
8. Maintain objectivity and professional judgment

Generate a comprehensive feasibility study that addresses all technical and economic aspects. The analysis should be detailed, realistic, and provide clear recommendations for project approval or rejection based on the feasibility assessment.
"""

In [29]:
output_parser = JsonOutputParser(pydantic_object=FeasibilityStudySchema)
def get_user_stories_from_llm(project_description):
    user_query =  f"Create a feasibility study for the this project {project_description}"
    chain = prompt | llm | output_parser
    response = chain.invoke({"system_prompt" : FEASIBILITY_STUDY_SYSTEM_PROMPT, "human_query" : user_query})
    return response

In [36]:
project_requirments = {
  "title": "E-commerce Platform for Handmade Goods",
  "description": "Develop a comprehensive e-commerce platform that connects artisans selling handmade goods directly with buyers. The platform will feature product listings, secure payment processing, order management, user profiles for both buyers and sellers, and a review system. The goal is to provide a seamless and trustworthy online marketplace that supports independent creators and offers unique products to consumers.",
  "project_duration_in_weeks": 24,
  "requirements": [
    {
      "requirement": "User registration and login for buyers and sellers",
      "priority": "critical",
      "type": "functional"
    },
    {
      "requirement": "Product listing creation and management for sellers",
      "priority": "critical",
      "type": "functional"
    },
    {
      "requirement": "Browse and search products by category, keywords, and filters",
      "priority": "high",
      "type": "functional"
    },
    {
      "requirement": "Shopping cart functionality",
      "priority": "critical",
      "type": "functional"
    },
    {
      "requirement": "Secure payment gateway integration (e.g., Stripe, PayPal)",
      "priority": "critical",
      "type": "functional"
    },
    {
      "requirement": "Order placement and tracking for buyers",
      "priority": "critical",
      "type": "functional"
    },
    {
      "requirement": "Order management and fulfillment for sellers",
      "priority": "high",
      "type": "functional"
    },
    {
      "requirement": "Product review and rating system",
      "priority": "medium",
      "type": "functional"
    },
    {
      "requirement": "User profile management (shipping addresses, payment methods)",
      "priority": "high",
      "type": "functional"
    },
    {
      "requirement": "Notifications for order status updates and messages",
      "priority": "medium",
      "type": "functional"
    },
    {
      "requirement": "Responsive design for desktop, tablet, and mobile devices",
      "priority": "critical",
      "type": "non_functional"
    },
    {
      "requirement": "Platform should handle 1000 concurrent users without performance degradation",
      "priority": "high",
      "type": "non_functional"
    },
    {
      "requirement": "Data encryption for sensitive user information",
      "priority": "critical",
      "type": "non_functional"
    },
    {
      "requirement": "Intuitive and clean user interface for product discovery and checkout",
      "priority": "high",
      "type": "user_interface"
    },
    {
      "requirement": "Admin panel for user, product, and order management",
      "priority": "high",
      "type": "functional"
    },
    {
      "requirement": "Email verification for new user registrations",
      "priority": "high",
      "type": "functional"
    },
    {
      "requirement": "Social media sharing options for products",
      "priority": "low",
      "type": "functional"
    },
    {
      "requirement": "Wishlist functionality for buyers",
      "priority": "nice_to_have",
      "type": "functional"
    },
    {
      "requirement": "Real-time chat functionality between buyers and sellers",
      "priority": "nice_to_have",
      "type": "functional"
    }
  ],
  "domain": "ecommerce",
  "prefered_techstack": "MERN_JS"
}

In [37]:
import json
project_requirements_json_string = json.dumps(project_requirments, indent=2)

In [38]:
feasibility_study = get_user_stories_from_llm(project_requirements_json_string)
feasibility_study

{'feasibility_study': {'project_title': 'E-commerce Platform for Handmade Goods',
  'technical_feasibility': {'classification': 'feasible',
   'details': {'technology_stack_analysis': {'compatibility_and_maturity': 'The MERN (MongoDB, Express.js, React.js, Node.js) stack is well-established and widely supported, making it suitable for this project.',
     'team_expertise': 'The team will need expertise in JavaScript, React for frontend, Node.js/Express for backend, and MongoDB for database management. The learning curve is moderate, but the stack is popular and well-documented.',
     'technical_dependencies': 'Third-party integrations include payment gateways (Stripe, PayPal), email services (e.g., SendGrid), and social media APIs. These integrations are common and well-supported.',
     'scalability_and_performance': 'The MERN stack is scalable and can handle the required 1000 concurrent users with proper optimization and hosting solutions.'},
    'resource_evaluation': {'team_size_a

In [39]:
print(feasibility_study)

{'feasibility_study': {'project_title': 'E-commerce Platform for Handmade Goods', 'technical_feasibility': {'classification': 'feasible', 'details': {'technology_stack_analysis': {'compatibility_and_maturity': 'The MERN (MongoDB, Express.js, React.js, Node.js) stack is well-established and widely supported, making it suitable for this project.', 'team_expertise': 'The team will need expertise in JavaScript, React for frontend, Node.js/Express for backend, and MongoDB for database management. The learning curve is moderate, but the stack is popular and well-documented.', 'technical_dependencies': 'Third-party integrations include payment gateways (Stripe, PayPal), email services (e.g., SendGrid), and social media APIs. These integrations are common and well-supported.', 'scalability_and_performance': 'The MERN stack is scalable and can handle the required 1000 concurrent users with proper optimization and hosting solutions.'}, 'resource_evaluation': {'team_size_and_skills': 'A team of 8